In [2]:
from torch.utils.data import Dataset
import numpy as np
import os

import sqlite3
import torch
import io
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device", device)
print(torch.cuda.get_device_name(0))
print(torch.cuda.current_device())

torch.cuda.current_device()
def adapt_array(arr):
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

def position_reformed(data, startgoal):
    datax = (data[0]-startgoal[0])*(3/2) + startx
    datay = (data[1]-startgoal[1])*(3/2) + starty
    return datax, datay

# Converts np.array to TEXT when inserting
sqlite3.register_adapter(np.ndarray, adapt_array)
# Converts TEXT to np.array when selecting
sqlite3.register_converter("array", convert_array)

class TrafficDataset(Dataset):
    def __init__(self, dbpath, train=True, ratio_test=0.8, num_sce=100, num_data=25):
        self.con = sqlite3.connect(dbpath, detect_types=sqlite3.PARSE_DECLTYPES)
        self.path = dbpath
        self.cur = self.con.cursor()
        self.cur.execute("select id from minicity")
        self.idlist = self.cur.fetchall()
        self.cur.execute("select data from minicity where id = " +  str(1))
        self.dataperow = int(len(self.cur.fetchone()[0]) / 2)
        numTrain = int(ratio_test * len(self.idlist))
        if (train):
            self.filelist = self.idlist[:numTrain]
        else:
            self.filelist = self.idlist[numTrain:]
            
    def __len__(self):
        return len(self.idlist) * self.dataperow
    
    def __getitem__(self, idx):
        rowid = idx // self.dataperow
        dataid = idx % self.dataperow
        self.cur.execute("select id, startgoal, occ, data, egoid from minicity where id = " +  str(rowid+1))
        results = self.cur.fetchone()
        start_goal = results[1].astype(np.single)
        observed = results[2].astype(np.single)
        traj = results[3][0:self.dataperow].astype(np.single)
        time_prob = results[3][self.dataperow].astype(np.single)[5]
        data = results[3][self.dataperow + dataid].astype(np.single)
        egoid = results[4]
        
        start_goal[4] = start_goal[4] - start_goal[0]
        start_goal[5] = start_goal[5] - start_goal[1]
        
        traj[:, 0] = traj[:, 0] - start_goal[0]
        traj[:, 1] = traj[:, 1] - start_goal[1]
        data[0] = data[0] - start_goal[0]
        data[1] = data[1] - start_goal[1]
        traj = traj[:,0:4]/50
        data = data[0:4]/50
        
        for i in range(0, int(len(observed)/2)):
            observed[i, :] = observed[2*i, :, ]
        
        observed = observed[0:int(len(observed)/2), :]
        start_goal[0] = 0
        start_goal[1] = 0
        start_goal = start_goal/50
        
        sample = {'start_goal': start_goal,
                              'traj': traj,
                             'observation': observed,
                             'data': data,
                             'egoid': egoid,
                             'timeprob':time_prob}
        return sample

Using device cuda:0
P106-100
0


In [4]:
import torch
import numpy as np

In [5]:
from torch.utils.data import Dataset, DataLoader

dbpath = '/home/rong/disk/database/minicity.db'
bs = 12
train_loader = DataLoader(TrafficDataset(dbpath = dbpath,
                            train = True),
                         batch_size = bs, shuffle=True, drop_last = True)
test_loader = DataLoader(TrafficDataset(dbpath = dbpath,
                            train = False),
                          batch_size = bs, shuffle=True, drop_last = True)

# batch = next(iter(train_loader))
# batch['data']

In [6]:
import torch.nn.functional as F
from torch import nn, optim

class convVAE(nn.Module):
    def __init__(self, sample_size, traj_size, cnnout_size, cond_out_size, encoder_layer_sizes, latent_size, decoder_layer_sizes):
        super(convVAE, self).__init__()

        assert type(encoder_layer_sizes) == list
        assert type(latent_size) == int
        assert type(decoder_layer_sizes) == list
        
        self.latent_size = latent_size
        self.condnn = CondNN(sample_size, traj_size, cnnout_size)
        self.encoder = Encoder(sample_size + cond_out_size, encoder_layer_sizes, latent_size)
        self.decoder = Decoder(latent_size +cond_out_size, decoder_layer_sizes, sample_size)

    def encode(self, x):
        return self.encoder(x)

    def decode(self, x):
        return self.decoder(x)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x, startend, traj, occ):
        c, _= self.condnn(startend, traj, occ)
#         print("x size: ", x.shape)
#         print("c size", c.shape)
        mu, logvar = self.encode(torch.cat((x, c), dim=1))
        z = self.reparameterize(mu, logvar)
        return self.decode(torch.cat((z, c), dim=-1)), mu, logvar
    
    def inference(self, startend, traj, occ, num_viz):
        c, alpha = self.condnn(startend, traj, occ)
        z = torch.randn(num_viz, self.latent_size, device = c.device)
        return self.decode(torch.cat((z, c), dim=-1)), alpha
    
class Encoder(nn.Module):
    def __init__(self, input_size, layer_sizes, latent_size):
        super(Encoder, self).__init__()

        layer_sizes = [input_size] + layer_sizes
        modules = []
        for i, (in_size, out_size) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
            modules.append(nn.Linear(in_size, out_size))
            modules.append(nn.ReLU())
#             modules.append(nn.Dropout(p=0.5))

        self.sequential = nn.Sequential(*modules)
        self.linear_means = nn.Linear(layer_sizes[-1], latent_size)
        self.linear_log_var = nn.Linear(layer_sizes[-1], latent_size)

    def forward(self, x):
        x = self.sequential(x)
        means = self.linear_means(x)
        log_vars = self.linear_log_var(x)
        return means, log_vars


class Decoder(nn.Module):
    def __init__(self, input_size, layer_sizes, sample_size):
        super(Decoder, self).__init__()

        layer_sizes = [input_size] + layer_sizes
        modules = []
        for i, (in_size, out_size) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
            modules.append(nn.Linear(in_size, out_size))
            modules.append(nn.ReLU())
#             modules.append(nn.Dropout(p=0.5))
        modules.append(nn.Linear(layer_sizes[-1], sample_size))

        self.sequential = nn.Sequential(*modules)

    def forward(self, x):
        return self.sequential(x)


class CondNN(nn.Module):
    def __init__(self, sampleSize, traj_size, outSize, encoder_dim=64, attention_dim=64):
        super(CondNN, self).__init__()
        self.sampleSize = sampleSize
        self.cnn = Conv3d(cnn_out_size)
        self.Attention = Attention(cnn_encode_dim=encoder_dim, condition_dim=sampleSize * 2, attention_dim=attention_dim) # + 3 for position
        self.fc1 = nn.Linear(encoder_dim + sampleSize * 2 + traj_size, outSize)

    def forward(self, startend, traj, occ):
        cnn_encode = self.cnn(occ)
        batch_size = startend.size(0)
        attention_weighted_encoding, alpha = self.Attention(cnn_encode, startend)
        x = torch.cat((attention_weighted_encoding, startend, traj.view(batch_size, -1)), dim=-1)
#         print("condnn cated size:", x.shape)
        x = self.fc1(x)
        return x, alpha

class Conv3d(nn.Module):
    def __init__(self, cnn_out_size):
        super(Conv3d, self).__init__()

        self.adap_pool = nn.AdaptiveAvgPool3d((25, 200, 200))
        self.conv_layer1 = self._make_conv_layer(1, 16)
        self.conv_layer2 = self._make_conv_layer(16, 32)
#         self.conv_layer3 = self._make_conv_layer(64, 124)
        self.conv_layer5=nn.Conv3d(32, 64, kernel_size=(1, 3, 3), padding=0)
        
        self.adap_pool2 = nn.AdaptiveAvgPool3d((6, 20, 20))

    def _make_conv_layer(self, in_c, out_c):
        conv_layer = nn.Sequential(
        nn.Conv3d(in_c, out_c, kernel_size=(2, 3, 3), padding=0),
        nn.LeakyReLU(),
        nn.Conv3d(out_c, out_c, kernel_size=(2, 3, 3), padding=1),
        nn.LeakyReLU(),
        nn.MaxPool3d((2, 2, 2)),
        )
        return conv_layer

    def forward(self, x):
        x = self.adap_pool(x)
#         print(x.size())
        x = self.conv_layer1(x)
#         print(x.size())
        x = self.conv_layer2(x)
#         print(x.size())
        x=self.conv_layer5(x)
#         print(x.size())
        x = self.adap_pool2(x)
#         print("cnn out size",x.size())

        return x
    

class Attention(nn.Module):
    def __init__(self, cnn_encode_dim, condition_dim, attention_dim):
        super(Attention, self).__init__()
        self.encoder_att = nn.Linear(cnn_encode_dim + 3, attention_dim) #位置信息
        self.condition_att = nn.Linear(condition_dim, attention_dim)
        self.full_att = nn.Linear(attention_dim, 1)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        self.position = self.initPosition()
    
    def initPosition(self):
        x = np.linspace(0, 5, 6, dtype='float32')
        y = np.linspace(0, 19, 20, dtype='float32')
        t = np.linspace(0, 19, 20, dtype='float32')
        tv,xv, yv = np.meshgrid(t,x,y)
        xv, yv, tv = xv.reshape((1,-1, 1)), yv.reshape((1,-1, 1)), tv.reshape((1,-1, 1))
        position = torch.from_numpy(np.concatenate((xv, yv, tv), axis = 2))
        
        return position
        
        
    def forward(self, encoder_out, condition):
        batch_size = encoder_out.size(0)
        encoder_dim = encoder_out.size(1)      
        encoder_out = encoder_out.permute(0, 2, 3, 4, 1)
        encoder_out = encoder_out.view(batch_size, -1, encoder_dim)  # (batch_size, num_pixels, encoder_dim)
        self.position = self.position.to(encoder_out.device)
        
        self.position = self.position.expand(batch_size, self.position.shape[1], self.position.shape[2]) #存疑
        encoder_out_pos = torch.cat((encoder_out, self.position), dim = 2)
#         print("cat encoder_out: ", encoder_out_pos.shape)
        att1 = self.encoder_att(encoder_out_pos)
        att2 = self.condition_att(condition)# 依然不清晰
        att2 = att2.unsqueeze(1)
        att2 = att2.expand(batch_size, att1.shape[1], -1)
#         print("att1: ", att1.shape)
#         print("att2: ", att2.shape)
        
        att = self.relu(self.full_att(att1 + att2))
#         print("att: ", att.shape)
        
        alpha = self.softmax(att)
#         print("encoder_out: ", encoder_out.shape)
#         print("alpha shape: ", alpha.shape)
        attention_weighted_encoding = (encoder_out * alpha).sum(dim=1)
#         print("attention_weighted_encoding: ", attention_weighted_encoding.shape)
            
        return attention_weighted_encoding, alpha

In [7]:
import torch 

X_dim = 4
traj_size = 25 * 4
z_dim = 50
cnn_out_size = 300
cond_out_size = 300

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device", device)

model = convVAE(sample_size = X_dim, 
                  traj_size = traj_size,
                  cnnout_size = cnn_out_size, 
                  cond_out_size = cond_out_size, 
                  encoder_layer_sizes = [512,1024,512], 
                  latent_size = z_dim, 
                  decoder_layer_sizes = [512,1024,512]).to(device)
print(model)

Using device cuda:0
convVAE(
  (condnn): CondNN(
    (cnn): Conv3d(
      (adap_pool): AdaptiveAvgPool3d(output_size=(25, 200, 200))
      (conv_layer1): Sequential(
        (0): Conv3d(1, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(16, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer2): Sequential(
        (0): Conv3d(16, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(32, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer5): Conv3d(32, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1))
      (adap

In [8]:
def loss_fn(recon_x, x, w, mean, log_var):
    MSE = torch.mean((w.expand_as(x) * (recon_x-x)**2))
    KLD = - 0.0005 * torch.mean(torch.sum(1 + log_var - mean.pow(2) - log_var.exp(), 1))
    return MSE + KLD, MSE

optimizer = optim.Adam(model.parameters(), lr=5e-4)

In [9]:
def train(epoch, writer):
    model.train()
    train_loss = 0
    mse_loss = 0
    w = torch.tensor([8, 10, 2, 3], dtype=torch.float).to(device)
    adap_pool = nn.AdaptiveAvgPool3d((25,200, 200))
    
    for batch_idx, batch in enumerate(train_loader):
        startgoal = batch["start_goal"].to(device)
        occ = batch["observation"]
        occ = occ[:, :, 200:600, 200:600]        
        occ = adap_pool(occ)
        occ = occ.to(device)
        occ = occ.unsqueeze(1)
        data = batch["data"].to(device)
        traj = batch["traj"].to(device)

        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data, startgoal, traj,  occ)
        loss, mse= loss_fn(recon_batch, data, w, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        mse_loss += mse.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item()))
        
            writer.add_scalar('BatchLoss/loss', loss.item(), batch_idx)
            writer.add_scalar('BatchLoss/mse_loss', mse.item(), batch_idx)

    epoch_loss = train_loss * len(data) / len(train_loader.dataset)
    epoch_mse = mse_loss * len(data) / len(train_loader.dataset)
    print('====> Epoch: {} Average loss: {:.7f}'.format(
          epoch, epoch_loss))
    return epoch, epoch_loss, epoch_mse

def test(epoch):
    model.eval()
    test_loss = 0
    mse_loss = 0
    w = torch.tensor([8, 10, 2, 3], dtype=torch.float).to(device)
    for batch_idx, batch in enumerate(test_loader):
        startgoal = batch["start_goal"].to(device)
        occ = batch["observation"].to(device)
        occ = occ.unsqueeze(1)
        data = batch["data"].to(device)
        
        recon_batch, mu, logvar = model(sample, startend, occ)
        loss, mse= loss_fn(recon_batch, data, w, mu, logvar)
        test_loss += loss.item()
        mse_loss += mse.item()

    epoch_loss = test_loss * len(data) / len(test_loader.dataset)
    epoch_mse = mse_loss * len(data) / len(test_loader.dataset)
    print('====> Epoch: {} Average test loss: {:.7f}'.format(
          epoch, epoch_loss))
    return epoch, epoch_loss, epoch_mse

In [12]:
epoch = 0
from torch.utils.tensorboard import SummaryWriter

# Writer will output to ./runs/ directory by default
writer = SummaryWriter('runs/minicity_attention_7_3')

In [13]:
for epoch in range(epoch, epoch + 200):
    log_interval = 20
    epoch, epoch_loss, epoch_mse = train(epoch, writer)
#     vis.line(X=torch.ones((1,1)).cpu()*epoch,Y=torch.Tensor([epoch_loss]).unsqueeze(0).cpu(),win=loss_window,update='append',name='loss')
#         vis.line(X=torch.ones((1,1)).cpu()*epoch,Y=torch.Tensor([epoch_mse]).unsqueeze(0).cpu(),win=loss_window,update='append',name='mse_loss')
    writer.add_scalar('Loss/loss', epoch_loss, epoch)
    writer.add_scalar('Loss/mse_loss', epoch_mse, epoch)

Train Epoch: 0 [0/210000 (0%)]	Loss: 0.070153
Train Epoch: 0 [240/210000 (0%)]	Loss: 0.056549


KeyboardInterrupt: 

In [45]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, 'checkpoints/minicity_attention_7.pt')

In [9]:
torch.cuda.empty_cache()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device", device)

model = convVAE(sample_size = X_dim, 
                  traj_size = traj_size,
                  cnnout_size = cnn_out_size, 
                  cond_out_size = cond_out_size, 
                  encoder_layer_sizes = [512,1024,512], 
                  latent_size = z_dim, 
                  decoder_layer_sizes = [512,1024,512]).to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)

checkpoint = torch.load('checkpoints/minicity_attention_7.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

model.eval()
print(model)

Using device cuda:0
convVAE(
  (condnn): CondNN(
    (cnn): Conv3d(
      (adap_pool): AdaptiveAvgPool3d(output_size=(25, 200, 200))
      (conv_layer1): Sequential(
        (0): Conv3d(1, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(16, 16, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer2): Sequential(
        (0): Conv3d(16, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1))
        (1): LeakyReLU(negative_slope=0.01)
        (2): Conv3d(32, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (3): LeakyReLU(negative_slope=0.01)
        (4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
      )
      (conv_layer5): Conv3d(32, 64, kernel_size=(1, 3, 3), stride=(1, 1, 1))
      (adap

In [88]:
test_data = test_loader.dataset
viz_idx =   torch.randint(0,len(test_data),[1]).item()  
#  变道场景idx
# viz_idx = 8753 弯道
# viz_idx = 143300 弯道
# viz_idx = 5107 直行
# viz_idx = 2984 转盘
# viz_idx = 101538 转盘
# viz_idx = 153309 阻塞
# viz_idx = 109779 变道

print(viz_idx)

batch = test_data[viz_idx]
startgoal = torch.from_numpy(batch["start_goal"]).to(device)
occ = torch.from_numpy(batch["observation"])
occ = occ[:, 200:600, 200:600]        
occ = occ.unsqueeze(0)
occ = occ.unsqueeze(1)
adap_pool = nn.AdaptiveAvgPool3d((25,200, 200))
occ = adap_pool(occ)
occ = occ.to(device)
traj = torch.from_numpy(batch["traj"]).to(device)
print(traj.shape)
data = torch.from_numpy(batch["data"]).to(device)

with torch.no_grad():
    model.eval()
    y_viz = torch.randn(1,4).to(device)
    for i in range(0, 10):
        num_viz = 8
        y_viz_p, alpha = model.inference(startgoal.expand(num_viz, X_dim * 2).to(device), traj.expand(num_viz, 25, 4),
                                occ.expand(num_viz, 1, -1, -1, -1).to(device), num_viz)
        torch.cuda.empty_cache()
        y_viz = torch.cat((y_viz_p, y_viz), dim = 0)

y_viz=y_viz.cpu().detach().numpy()*50
occ=occ.cpu().detach().numpy()
startgoal=startgoal.cpu().detach().numpy() * 50
data=data.cpu().detach().numpy() * 50
alpha=alpha.cpu().detach().numpy()
torch.cuda.empty_cache()
# from utils.NarrowPassage import plotCondition, plotSample, plotSpeed, plotSampleAttention

%matplotlib
# from utils.HighWay import plotData, plotOrientSpeed, plotAlpha
    
y_viz=y_viz[:-1]
plotData(occ, startgoal, y_viz)
plotOrientSpeed(startgoal, y_viz)
# plotAlpha(alpha)

64486
torch.Size([25, 4])
Using matplotlib backend: Qt5Agg
[ 0.         0.        -3.1416     5.74      -8.9        5.1799927
 -4.2487     5.73     ]
70.33333460489908 82.73335774739583


In [83]:
plotAlpha(alpha)

In [70]:
import matplotlib.pyplot as plt

startx = 100
starty = 100

def position2imagep(data, startgoal):
    datax = (data[0])*(10/3) + 100
    datay = 100-(data[1])*(10/3) 
    return datax, datay

def plotCondition(occ, startgoal):
    fig = plt.figure()
    occ=occ.squeeze(0)
    occ=occ.squeeze(0)
    myobj = plt.imshow(occ[0, :, :])
    goalx, goaly = position2imagep(startgoal[4:], startgoal)
    plt.scatter(startx, starty)
    plt.scatter(goalx, goaly)
    for frame in occ:
        myobj.set_data(frame)
        plt.draw()
        plt.pause(0.1)

def plotData(occ, startgoal,data):
    fig = plt.figure()
    occ=occ.squeeze(0)
    occ=occ.squeeze(0)
    myobj = plt.imshow(occ[0, :, :])
    goalx, goaly = position2imagep(startgoal[4:], startgoal)
    print(startgoal)
    print(goalx, goaly)
    plt.scatter(startx, starty)
    plt.scatter(goalx, goaly)
    if len(data.shape) > 1:
        for row in data:
            datax, datay = position2imagep(row, startgoal)
            plt.scatter(datax, datay, marker='*', c='#d62728')
    else:
        datax, datay = position2imagep(data, startgoal)
        plt.scatter(datax, datay, marker='*', c='#d62728')
    for frame in occ:
        myobj.set_data(frame)
        plt.draw()
        plt.pause(0.1)

def plotOrientSpeed(startgoal, data):
    fig = plt.figure()
    goalx, goaly = position2imagep(startgoal[4:], startgoal)
    plt.scatter(data[:, 2], data[:,3])
    plt.scatter(startgoal[2], startgoal[3])
    plt.scatter(startgoal[6], startgoal[7])
    
def plotAlpha(alpha):
    fig = plt.figure()
    alpha = alpha[0,:]
    alpha=alpha.reshape(6,20,20)
    
    myobj = plt.imshow(alpha[0, :, :])

    for frame in alpha:
        myobj.set_data(frame)
        plt.draw()
        plt.pause(0.3)

In [18]:
test_data = test_loader.dataset
viz_idx =   torch.randint(0,len(test_data),[1]).item()  
#  变道场景idx
#  
print(viz_idx)

batch = test_data[viz_idx]
startgoal = torch.from_numpy(batch["start_goal"]).to(device)
occ = torch.from_numpy(batch["observation"])
occ = occ.unsqueeze(0)
occ = occ.unsqueeze(1)
# adap_pool = nn.AdaptiveAvgPool3d((25,100, 600))
# occ = adap_pool(occ)
occ = occ.to(device)
data = torch.from_numpy(batch["data"]).to(device)

occ=occ.cpu().detach().numpy()
startgoal=startgoal.cpu().detach().numpy()
data=data.cpu().detach().numpy()
torch.cuda.empty_cache()

plotData(occ, startgoal, data)

445954


In [19]:
occ.shape

(1, 1, 25, 300, 1853)